In [2]:
import pandas as pd
import numpy as np


/var/folders/nd/r7yjrrdn6m5g8vq_zr3vwbrr0000gn/T/ipykernel_14588/2937650959.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df_icu = pd.read_csv('mimic-iv-2.2/icu/icustays.csv.gz')
df_icd = pd.read_csv('mimic-iv-2.2/hosp/diagnoses_icd.csv.gz')
df_admission = pd.read_csv('mimic-iv-2.2/hosp/admissions.csv.gz')
df_patients = pd.read_csv('mimic-iv-2.2/hosp/patients.csv.gz')
df_med = pd.read_csv('mimic-iv-2.2/hosp/prescriptions.csv.gz')

/var/folders/nd/r7yjrrdn6m5g8vq_zr3vwbrr0000gn/T/ipykernel_14588/386627112.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_med = pd.read_csv('mimic-iv-2.2/hosp/prescriptions.csv.gz')


In [4]:
heart_failure_code = [
    str(4280), str(4281), str(42820), str(42821), str(42822), str(42823), str(42830), str(42831), str(42832), str(42833), str(42840), str(42841), str(42842), str(42843), str(4289),
    'I50', 'I502', 'I5020', 'I5021', 'I5022', 'I5023', 'I503', 'I5030', 'I5031', 'I5032', 'I5033', 'I504', 'I5040',
    'I5041', 'I5042', 'I5043', 'I508', 'I5081', 'I50810', 'I50811', 'I50812', 'I50813', 'I50814', 'I5082', 'I5083',
    'I5084', 'I5089', 'I509'
]
df_icd["icd_code"] = df_icd["icd_code"].astype(str)

In [5]:
grouped_icd = df_icd.groupby('subject_id')
grouped_icd = grouped_icd['icd_code'].apply(list)
filtered_grouped_icd = grouped_icd[grouped_icd.apply(lambda x: any(code in heart_failure_code for code in x))]
filtered_df_icu_noHF = df_icu[~df_icu['subject_id'].isin(filtered_grouped_icd.index)]



In [6]:
grouped_icd.index[0]

10000032

In [7]:
grouped_filtered_df_icu = filtered_df_icu_noHF.groupby('subject_id').apply(lambda x: x.loc[x['intime'].idxmax()])
selected_columns = grouped_filtered_df_icu[['subject_id', "hadm_id", 'intime', 'outtime', 'los']]
selected_columns.reset_index(drop=True, inplace=True)
df_admission_filtered = df_admission[df_admission['hadm_id'].isin(selected_columns['hadm_id'])]
df_admission_filtered = df_admission_filtered[["subject_id", "hadm_id", "admittime", "dischtime", "deathtime", "insurance", "language", "marital_status", "race"]]
merged_df = df_admission_filtered.merge(selected_columns, on=['hadm_id', 'subject_id'])
merged_df


/var/folders/nd/r7yjrrdn6m5g8vq_zr3vwbrr0000gn/T/ipykernel_14588/1996727992.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_filtered_df_icu = filtered_df_icu_noHF.groupby('subject_id').apply(lambda x: x.loc[x['intime'].idxmax()])


,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,language,marital_status,race,intime,outtime,los
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,NaN,Other,?,MARRIED,WHITE,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
2,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaN,Other,ENGLISH,MARRIED,WHITE,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
3,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817
4,10002348,22725460,2112-11-30 22:22:00,2112-12-10 17:56:00,NaN,Medicare,ENGLISH,SINGLE,WHITE,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512
...,...,...,...,...,...,...,...,...,...,...,...,...
36406,19999442,26785317,2148-11-19 10:00:00,2148-12-04 16:25:00,NaN,Medicaid,ENGLISH,DIVORCED,WHITE,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
36407,19999625,25304202,2139-10-10 18:06:00,2139-10-16 03:30:00,NaN,Medicare,?,MARRIED,WHITE,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
36408,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,Other,ENGLISH,SINGLE,WHITE,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
36409,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,Other,ENGLISH,WIDOWED,WHITE,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [8]:
df_patients_filtered = df_patients[df_patients['subject_id'].isin(merged_df['subject_id'])]
final_df = merged_df.merge(df_patients_filtered, on='subject_id')

In [9]:
final_df['admittime'] = pd.to_datetime(final_df['admittime'])
final_df['dischtime'] = pd.to_datetime(final_df['dischtime'])
final_df['deathtime'] = pd.to_datetime(final_df['deathtime'])
final_df['dod'] = pd.to_datetime(final_df['dod'])
final_df['dischtime'] - final_df['admittime']
final_df["dischtime_since_admission"] = final_df['dischtime'] - final_df['admittime']
final_df["dod_since_disch"] = final_df['dod'] - final_df['dischtime']

# Convert to days
final_df["dischtime_since_admission"] = final_df["dischtime_since_admission"] / pd.Timedelta(days=1)
final_df["dod_since_disch"] = final_df["dod_since_disch"] / pd.Timedelta(days=1)

final_df["censoring_time"] = 365

final_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,language,marital_status,race,intime,outtime,los,gender,anchor_age,anchor_year,anchor_year_group,dod,dischtime_since_admission,dod_since_disch,censoring_time
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,F,52,2180,2014 - 2016,2180-09-09,2.222222,45.253472,365
1,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,Other,?,MARRIED,WHITE,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,F,55,2157,2011 - 2013,NaT,5.914583,NaN,365
2,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,Other,ENGLISH,MARRIED,WHITE,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,F,46,2110,2011 - 2013,NaT,2.994444,NaN,365
3,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,F,68,2122,2008 - 2010,2131-01-20,12.358333,-0.218750,365
4,10002348,22725460,2112-11-30 22:22:00,2112-12-10 17:56:00,NaT,Medicare,ENGLISH,SINGLE,WHITE,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512,F,77,2112,2017 - 2019,NaT,9.815278,NaN,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36406,19999442,26785317,2148-11-19 10:00:00,2148-12-04 16:25:00,NaT,Medicaid,ENGLISH,DIVORCED,WHITE,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,M,41,2146,2008 - 2010,NaT,15.267361,NaN,365
36407,19999625,25304202,2139-10-10 18:06:00,2139-10-16 03:30:00,NaT,Medicare,?,MARRIED,WHITE,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,M,81,2138,2008 - 2010,NaT,5.391667,NaN,365
36408,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,Other,ENGLISH,SINGLE,WHITE,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,F,46,2147,2017 - 2019,NaT,10.011111,NaN,365
36409,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,Other,ENGLISH,WIDOWED,WHITE,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,M,58,2164,2008 - 2010,2164-09-17,6.996528,-0.570833,365


In [10]:
import re
hypertension_codes = [r'401.\w', 'I10']
atrial_fibrillation_codes = ['427.31', 'I48.91']
type1_diabetes_codes = [r'250.\w1', r"E10.\w"]
type2_diabetes_codes = [r'250.\w0', r'E11.\w']
copd_codes = ['496', r'J44.\w']
asthma_codes = [r'493.\w\w', r'J45.\w']
liver_disease_codes = [r'571.\w', 'K70", "K71", "K72', 'K73', 'K74', 'K75', 'K76', 'K77']
chronic_kidney_disease_codes = [r'585.\w', r'N18.\w']

list1 = [str(i) for i in range(140, 240)]
list2 = [f"C{i:02d}" for i in range(0, 100)]
list3 = [f"D{i:02d}" for i in range(0, 50)]
cancer_codes_1 = list1 + list2 + list3

depression_codes = [r'296.2\w', r'296.3\w', '311', r'F32.\w', r'F33.\w']
osteoarthritis_codes = [r'715.\w\w', "M15", "M16", "M17", "M18", "M19"]

list1 = [str(i) for i in range(280, 286)]
list2 = [f"D{i:02d}" for i in range(50, 65)]
anemia_codes_1 = list1 + list2

hypertension_codes = [r'401.\w']
hypertension_codes_no_w = ['I10']

atrial_fibrillation_codes = [r'I48.\w', r"I48.\w\w"]
atrial_fibrillation_codes_no_w = ['427.31']

type1_diabetes_codes = [r'250.\w1', r"E10.\w"]
type1_diabetes_codes_no_w = []

type2_diabetes_codes = [r'250.\w0', r'E11.\w']
type2_diabetes_codes_no_w = []

copd_codes = [r'J44.\w']
copd_codes_no_w = ['496']

asthma_codes = [r'493.\w\w', r'J45.\w']
asthma_codes_no_w = []

liver_disease_codes = [r'571.\w']
liver_disease_codes_no_w = ['K70", "K71", "K72', 'K73', 'K74', 'K75', 'K76', 'K77']

chronic_kidney_disease_codes = [r'585.\w', r'N18.\w', r"403.\w\w", r"404.\w\w"]
chronic_kidney_disease_codes_no_w = ["N18"]

cancer_codes = []
cancer_codes_no_w = cancer_codes_1

depression_codes = [r'296.2\w', r'296.3\w', r'F32.\w', r'F33.\w']
depression_codes_no_w = ['311']

osteoarthritis_codes = [r'715.\w\w']
osteoarthritis_codes_no_w = ["M15", "M16", "M17", "M18", "M19"]

anemia_codes = []
anemia_codes_no_w = anemia_codes_1


In [11]:
import re
def check_string_pattern(patterns, not_patterns, strings):
    for string in strings:
        for pattern in patterns:
            if re.search(pattern, string):
                return True
    if any(code in strings for code in not_patterns):
        return True
    return False

In [12]:
# Assuming subject_id is a column in both final_df and grouped_icd
ls = final_df['subject_id'].isin(grouped_icd.index)
false_indices = np.where(ls == False)[0]
final_df = final_df.drop(false_indices)

In [13]:
final_df['has_type1_diabetes'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(type1_diabetes_codes, type1_diabetes_codes_no_w, grouped_icd[x])]))
final_df['has_type2_diabetes'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(type2_diabetes_codes,type2_diabetes_codes_no_w ,grouped_icd[x])]))
final_df['has_hypertension'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(hypertension_codes, hypertension_codes_no_w ,grouped_icd[x])]))
final_df['has_atrial_fibrillation'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(atrial_fibrillation_codes, atrial_fibrillation_codes_no_w,grouped_icd[x])]))
final_df['has_copd'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(copd_codes, copd_codes_no_w,grouped_icd[x])]))
final_df['has_asthma'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(asthma_codes, asthma_codes_no_w,grouped_icd[x])]))
final_df['has_liver_disease'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(liver_disease_codes,liver_disease_codes_no_w, grouped_icd[x])]))
final_df['has_chronic_kidney_disease'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(chronic_kidney_disease_codes, chronic_kidney_disease_codes_no_w,grouped_icd[x])]))
final_df['has_cancer'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(cancer_codes, cancer_codes_no_w, grouped_icd[x])]))
final_df['has_depression'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(depression_codes, depression_codes_no_w,grouped_icd[x])]))
final_df['has_osteoarthritis'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(osteoarthritis_codes, osteoarthritis_codes_no_w,grouped_icd[x])]))
final_df['has_anemia'] = final_df['subject_id'].apply(lambda x: any([check_string_pattern(anemia_codes,anemia_codes_no_w, grouped_icd[x])]))



In [14]:
disease_columns = ['has_hypertension', 'has_atrial_fibrillation', 'has_type1_diabetes', 'has_type2_diabetes', 'has_copd', 'has_asthma', 'has_liver_disease', 'has_chronic_kidney_disease', 'has_cancer', 'has_depression', 'has_osteoarthritis', 'has_anemia']
average_frequency = final_df[disease_columns].mean()
average_frequency

has_hypertension              0.260212
has_atrial_fibrillation       0.070705
has_type1_diabetes            0.010630
has_type2_diabetes            0.060761
has_copd                      0.052108
has_asthma                    0.046669
has_liver_disease             0.002610
has_chronic_kidney_disease    0.003296
has_cancer                    0.018624
has_depression                0.116907
has_osteoarthritis            0.000000
has_anemia                    0.107458
dtype: float64

In [15]:
medication_lst = ["Enalapril", "Lisinopril", "Ramipril", "Carvedilol", "Metoprolol Succinate", "Bisoprolol", "Furosemide", "Bumetanide", "Spironolactone", "Warfarin", "Apixaban", "Rivaroxaban"]
grouped_med = df_med.groupby('subject_id')
grouped_med = grouped_med['drug'].apply(list)
filtered_grouped_med = grouped_med[grouped_med.apply(lambda x: any(code in medication_lst for code in x))]
for medication in medication_lst:
    final_df[medication] = final_df['subject_id'].apply(lambda x: medication in grouped_med.get(x, []))
average_frequency = final_df[medication_lst].mean()
average_frequency

Enalapril               0.000000
Lisinopril              0.218514
Ramipril                0.002170
Carvedilol              0.025958
Metoprolol Succinate    0.000027
Bisoprolol              0.000000
Furosemide              0.421481
Bumetanide              0.003324
Spironolactone          0.042467
Warfarin                0.136327
Apixaban                0.028760
Rivaroxaban             0.016591
dtype: float64

In [16]:
final_df["ACE Inhibitor"] = final_df["Enalapril"] | final_df["Lisinopril"] | final_df["Ramipril"]
final_df["Beta Blocker"] = final_df["Carvedilol"] | final_df["Metoprolol Succinate"] | final_df["Bisoprolol"]
final_df["Diuretic"] = final_df["Furosemide"] | final_df["Bumetanide"] | final_df["Spironolactone"]
final_df["Anticoagulant"] = final_df["Warfarin"] | final_df["Apixaban"] | final_df["Rivaroxaban"]

average_frequency = final_df[["ACE Inhibitor", "Beta Blocker", "Diuretic", "Anticoagulant"]].mean()
average_frequency

ACE Inhibitor    0.220327
Beta Blocker     0.025985
Diuretic         0.427936
Anticoagulant    0.170169
dtype: float64

In [17]:
final_df

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,language,marital_status,race,intime,...,Furosemide,Bumetanide,Spironolactone,Warfarin,Apixaban,Rivaroxaban,ACE Inhibitor,Beta Blocker,Diuretic,Anticoagulant
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 14:00:00,...,True,False,True,False,False,False,False,False,True,False
1,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,Other,?,MARRIED,WHITE,2157-12-19 15:42:24,...,False,False,False,False,False,False,False,False,False,False
2,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,Other,ENGLISH,MARRIED,WHITE,2110-04-11 15:52:22,...,False,False,True,False,False,False,False,False,True,False
3,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-11 04:20:05,...,True,False,False,False,True,True,False,False,True,True
4,10002348,22725460,2112-11-30 22:22:00,2112-12-10 17:56:00,NaT,Medicare,ENGLISH,SINGLE,WHITE,2112-11-30 23:24:00,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36406,19999442,26785317,2148-11-19 10:00:00,2148-12-04 16:25:00,NaT,Medicaid,ENGLISH,DIVORCED,WHITE,2148-11-19 14:23:43,...,False,False,False,False,False,False,False,False,False,False
36407,19999625,25304202,2139-10-10 18:06:00,2139-10-16 03:30:00,NaT,Medicare,?,MARRIED,WHITE,2139-10-10 19:18:00,...,False,False,False,False,False,False,False,False,False,False
36408,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,Other,ENGLISH,SINGLE,WHITE,2149-01-08 18:12:00,...,False,False,False,False,False,False,False,False,False,False
36409,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,Other,ENGLISH,WIDOWED,WHITE,2164-09-12 09:26:28,...,False,False,False,False,False,False,False,False,False,False


In [18]:
final_df_with_dod = final_df[final_df['dod'].notnull()]
final_df_with_dod

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,language,marital_status,race,intime,...,Furosemide,Bumetanide,Spironolactone,Warfarin,Apixaban,Rivaroxaban,ACE Inhibitor,Beta Blocker,Diuretic,Anticoagulant
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 14:00:00,...,True,False,True,False,False,False,False,False,True,False
3,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-11 04:20:05,...,True,False,False,False,True,True,False,False,True,True
6,10002930,25922998,2198-04-17 19:38:00,2198-04-22 16:02:00,NaT,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2198-04-17 21:24:00,...,False,False,False,False,False,False,False,False,False,False
11,10004720,22081550,2186-11-12 18:01:00,2186-11-17 18:30:00,2186-11-17 18:30:00,Medicare,ENGLISH,SINGLE,WHITE,2186-11-12 19:55:00,...,False,False,False,False,False,False,False,False,False,False
17,10005866,20364112,2149-10-01 18:56:00,2149-10-25 18:50:00,NaT,Medicaid,ENGLISH,SINGLE,PORTUGUESE,2149-10-02 12:48:08,...,True,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36401,19998843,24842066,2187-02-05 09:27:00,2187-02-08 17:28:00,NaT,Other,?,SINGLE,UNKNOWN,2187-02-05 10:12:00,...,False,False,False,False,False,False,False,False,False,False
36402,19998878,26489544,2132-09-30 20:06:00,2132-10-02 12:00:00,NaT,Medicare,ENGLISH,SINGLE,WHITE,2132-09-30 21:55:00,...,True,False,True,False,False,False,False,False,True,False
36404,19999287,20175828,2197-08-03 20:58:00,2197-08-18 15:37:00,NaT,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2197-08-04 00:02:00,...,False,False,False,False,False,False,False,False,False,False
36405,19999297,21439025,2162-08-14 23:55:00,2162-08-23 04:16:00,2162-08-23 04:16:00,Other,ENGLISH,SINGLE,MULTIPLE RACE/ETHNICITY,2162-08-16 05:48:32,...,False,False,False,False,False,False,False,False,False,False


In [22]:
selected_rows = final_df[final_df["dod"] - final_df["dischtime"] <= pd.Timedelta(days=1)]
selected_rows

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,language,marital_status,race,intime,...,Furosemide,Bumetanide,Spironolactone,Warfarin,Apixaban,Rivaroxaban,ACE Inhibitor,Beta Blocker,Diuretic,Anticoagulant
3,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-11 04:20:05,...,True,False,False,False,True,True,False,False,True,True
11,10004720,22081550,2186-11-12 18:01:00,2186-11-17 18:30:00,2186-11-17 18:30:00,Medicare,ENGLISH,SINGLE,WHITE,2186-11-12 19:55:00,...,False,False,False,False,False,False,False,False,False,False
19,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,Medicaid,ENGLISH,NaN,UNKNOWN,2111-11-14 00:19:12,...,False,False,False,False,False,False,False,False,False,False
25,10007818,22987108,2146-06-10 16:37:00,2146-07-12 00:00:00,2146-07-12 20:50:00,Medicare,ENGLISH,MARRIED,WHITE,2146-06-22 11:46:29,...,False,False,False,False,False,False,False,False,False,False
39,10011365,26712576,2166-02-11 14:09:00,2166-02-20 00:20:00,2166-02-20 00:20:00,Medicare,ENGLISH,WIDOWED,WHITE,2166-02-11 16:03:00,...,True,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36390,19995127,27369164,2138-06-06 18:00:00,2138-06-12 01:48:00,2138-06-12 01:48:00,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2138-06-07 07:15:05,...,False,False,False,False,False,False,False,False,False,False
36399,19997886,20793010,2186-11-12 07:10:00,2186-12-10 20:35:00,NaT,Other,ENGLISH,WIDOWED,WHITE,2186-12-06 10:12:48,...,True,False,True,False,False,False,False,False,True,False
36401,19998843,24842066,2187-02-05 09:27:00,2187-02-08 17:28:00,NaT,Other,?,SINGLE,UNKNOWN,2187-02-05 10:12:00,...,False,False,False,False,False,False,False,False,False,False
36405,19999297,21439025,2162-08-14 23:55:00,2162-08-23 04:16:00,2162-08-23 04:16:00,Other,ENGLISH,SINGLE,MULTIPLE RACE/ETHNICITY,2162-08-16 05:48:32,...,False,False,False,False,False,False,False,False,False,False


In [23]:
final_df_noICUdeath = final_df[~final_df["subject_id"].isin(selected_rows["subject_id"])]
final_df_noICUdeath

,subject_id,hadm_id,admittime,dischtime,deathtime,insurance,language,marital_status,race,intime,...,Furosemide,Bumetanide,Spironolactone,Warfarin,Apixaban,Rivaroxaban,ACE Inhibitor,Beta Blocker,Diuretic,Anticoagulant
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 14:00:00,...,True,False,True,False,False,False,False,False,True,False
1,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,Other,?,MARRIED,WHITE,2157-12-19 15:42:24,...,False,False,False,False,False,False,False,False,False,False
2,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,Other,ENGLISH,MARRIED,WHITE,2110-04-11 15:52:22,...,False,False,True,False,False,False,False,False,True,False
4,10002348,22725460,2112-11-30 22:22:00,2112-12-10 17:56:00,NaT,Medicare,ENGLISH,SINGLE,WHITE,2112-11-30 23:24:00,...,False,False,False,False,False,False,True,False,False,False
5,10002443,21329021,2183-10-17 23:20:00,2183-10-20 18:47:00,NaT,Other,ENGLISH,SINGLE,WHITE,2183-10-18 00:47:00,...,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36404,19999287,20175828,2197-08-03 20:58:00,2197-08-18 15:37:00,NaT,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2197-08-04 00:02:00,...,False,False,False,False,False,False,False,False,False,False
36406,19999442,26785317,2148-11-19 10:00:00,2148-12-04 16:25:00,NaT,Medicaid,ENGLISH,DIVORCED,WHITE,2148-11-19 14:23:43,...,False,False,False,False,False,False,False,False,False,False
36407,19999625,25304202,2139-10-10 18:06:00,2139-10-16 03:30:00,NaT,Medicare,?,MARRIED,WHITE,2139-10-10 19:18:00,...,False,False,False,False,False,False,False,False,False,False
36408,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,Other,ENGLISH,SINGLE,WHITE,2149-01-08 18:12:00,...,False,False,False,False,False,False,False,False,False,False


In [24]:
final_df_noICUdeath.to_csv('final_df_noHF.csv', index=False)